# Ontology Search

This program will load and explore the Cell Ontology from https://www.ebi.ac.uk/ols/ontologies/cl

From the above site, you can download the ontology file as *cl.owl*.

The leafs in the ontology file contain information from the following:

- BFO: Basic Formal Ontology 
- CARO: Common Anatomy Reference Ontology
- CHEBI: Chemical Entities of Biological Interest
- CL: Cell Ontology
- CP: unclear meaning
- http: start of website, inclusion is unclear
- GO: Gene Ontology
- NCBITaxon: ncbitaxon
- PATO: Phenotype and Trait Ontology
- PR: Protein Ontology
- RO: Relations Ontology
- UBERON: Uber-anatomy ontology



To acess the ontology, we will use *pronto*, which can be installed with *pip*. Some resources:

- From https://github.com/althonos/pronto
- DOI: https://doi.org/10.5281/zenodo.595572
- PDF of release notes: https://pronto.readthedocs.io/_/downloads/en/latest/pdf/

In [1]:
from pronto import Ontology

import json
import urllib.request

import pandas as pd

In [2]:
cl = Ontology.from_obo_library('cl.owl')

/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/python3.8/site-packages/pronto/parsers/rdfxml.py:289: SyntaxWarning: <Element '{http://purl.org/dc/elements/1.1/}description' at 0x7f8cb0ce9720> contains text but no `xsd:datatype`
  meta.annotations.add(self._extract_literal_pv(child))
/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/python3.8/site-packages/pronto/parsers/rdfxml.py:289: SyntaxWarning: <Element '{http://purl.org/dc/elements/1.1/}title' at 0x7f8cb0ce9860> contains text but no `xsd:datatype`
  meta.annotations.add(self._extract_literal_pv(child))
/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/python3.8/site-packages/pronto/parsers/rdfxml.py:289: SyntaxWarning: <Element '{http://www.w3.org/2002/07/owl#}versionInfo' at 0x7f8cb0ce9e00> contains text but no `xsd:datatype`
  meta.annotations.add(self._extract_literal_pv(child))
/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/python3.8/site-packages/pronto/parsers/rdfxml.py:113: SyntaxWarning: unknown elem

/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/python3.8/site-packages/pronto/parsers/rdfxml.py:117: SyntaxWarning: unknown element in `owl:Class`: {http://purl.org/dc/terms/}contributor
  self._extract_term(class_, curies)
/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/python3.8/site-packages/pronto/parsers/rdfxml.py:117: SyntaxWarning: unknown element in `owl:Class`: {http://xmlns.com/foaf/0.1/}depicted_by
  self._extract_term(class_, curies)
/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/python3.8/site-packages/pronto/parsers/rdfxml.py:117: SyntaxWarning: unknown element in `owl:Class`: {http://www.geneontology.org/formats/oboInOwl#}note
  self._extract_term(class_, curies)
/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/python3.8/site-packages/pronto/parsers/rdfxml.py:117: SyntaxWarning: unknown element in `owl:Class`: {http://purl.obolibrary.org/obo/}IAO_0000119
  self._extract_term(class_, curies)
/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/py

/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/python3.8/site-packages/pronto/ontology.py:283: SyntaxWarning: unknown axiom property: 'http://purl.obolibrary.org/obo/UBPROP_0000001'
  cls(self).parse_from(_handle)  # type: ignore
/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/python3.8/site-packages/pronto/ontology.py:283: SyntaxWarning: unknown axiom property: 'http://purl.obolibrary.org/obo/UBPROP_0000003'
  cls(self).parse_from(_handle)  # type: ignore
/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/python3.8/site-packages/pronto/ontology.py:283: SyntaxWarning: unknown axiom property: 'http://purl.obolibrary.org/obo/UBPROP_0000012'
  cls(self).parse_from(_handle)  # type: ignore
/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/python3.8/site-packages/pronto/ontology.py:283: SyntaxWarning: unknown axiom property: 'http://purl.obolibrary.org/obo/UBPROP_0000008'
  cls(self).parse_from(_handle)  # type: ignore
/Users/josh.fuchs/My Drive/Personal/michigan/venv/li

In [3]:
cl

Ontology('http://purl.obolibrary.org/obo/cl.owl')

In [4]:
cl['CL:0002116']

Term('CL:0002116', name='B220-low CD38-positive unswitched memory B cell')

## Get tree structure

In [5]:
def find_parent(subclass, distance=1):
    '''
    Finds parent(s) of input subclass for current ontology, which must be loaded as cl. Parents will be 
    found within input distance of subclass. Immediate parents correspond with distance = 1. 
    
    Parameters
    ----------
    subclass : str
        string with full name of subclass you want to identify the parents of
        
    distance (optional): int
        distance from subclass you want to find parents of
        default = 1
        
    Returns
    -------
    parents : list
        list of parents for input subclass
    '''
    root_node = cl[subclass]
    parents = []
    for term in root_node.superclasses(distance=distance,with_self=False).to_set():
        parents.append(term.id)
    return parents

In [6]:
def get_tree(root,tree_dict,level=0):
    '''
    Determines tree structure starting with a leaf and identifying all superclasses/parents
    
    Parameters
    ----------
    root : str
        string with full name of ID you want to identify the parents of
        
    tree_dict : dictionary
        dictionary with tree. keys are levels, where level=0 is the initial leaf.
        values are ontology IDs
        
    level (optional): int
        level of root to build tree dictionary
        default = 0
        
    Returns
    -------
    tree_dict : dictionary
        dictionary of tree given inital root. 
        keys are levels, where level=0 is the initial leaf.
        values are ontology IDs
    '''


    # add root to dictionary if level = 0
    # need to check if level in tree_dict in case
    # we are iterating over multiple leafs
    if level == 0:
        if level in tree_dict:
            tree_dict[level].append(root)
        else:
            tree_dict[level] = [root]
    
    # get parent ids
    parents_of_id = find_parent(root)
    level += 1
    
    # add parents to dictionary
    if level in tree_dict:
        tree_dict[level] += parents_of_id
    else:
        tree_dict[level] = parents_of_id

    if 'CL:0000000' in parents_of_id: #parents_of_id is None:
        # we're at the final cell node
        pass
    else:
        for parent in parents_of_id:
            tree_dict = get_tree(parent,tree_dict,level=level)
    
    return tree_dict


In [31]:
tree_dict = {}
tree_dict = get_tree('CL:0000934',no_dup_dict,level=0)

# tree dict will have duplicates
# so let's remove them now

for key, value in tree_dict.items():
    tree_dict[key] = list(set(value))
    
tree_dict

{0: ['CL:0000934'],
 1: ['CL:0000624'],
 2: ['CL:0000791'],
 3: ['CL:0002419', 'CL:0000789'],
 4: ['CL:0000084', 'CARO:0000006', 'PR:000050567'],
 5: ['BFO:0000040', 'CL:0000542', 'CARO:0000000'],
 6: ['CL:0000842', 'BFO:0000004', 'CARO:0030000'],
 7: ['CL:0000738', 'BFO:0000004', 'BFO:0000002', 'CL:0000226'],
 8: ['CL:0000219', 'CL:0000988', 'CARO:0000000', 'CL:0002242', 'BFO:0000002'],
 9: ['CARO:0030000', 'CL:0002371', 'CL:0000548', 'CL:0000003'],
 10: ['CL:0000255', 'BFO:0000004', 'CL:0000000', 'CL:0000003'],
 11: ['BFO:0000002', 'CL:0000000', 'CL:0000003'],
 12: ['CL:0000000']}

In [32]:
leafs = ['CL:0000900', 'CL:0000934', 'CL:0001044']

leafs_dict_test = {}

for x in leafs:
    leafs_dict_test = get_tree(x,leafs_dict_test,level=0)

# leafs_dict_test will have duplicates
# so let's remove them now

for key, value in leafs_dict_test.items():
    leafs_dict_test[key] = list(set(value))
        
leafs_dict_test

{0: ['CL:0001044', 'CL:0000900', 'CL:0000934'],
 1: ['CL:0000625', 'CL:0000911', 'CL:0000898', 'CL:0000624'],
 2: ['CL:0002419', 'CL:0000791'],
 3: ['CL:0000084', 'CL:0002419', 'CL:0000789', 'CARO:0000006', 'PR:000050567'],
 4: ['CL:0000542',
  'CL:0000084',
  'BFO:0000040',
  'CARO:0000000',
  'CARO:0000006',
  'PR:000050567'],
 5: ['CL:0000542',
  'CARO:0030000',
  'CL:0000842',
  'BFO:0000040',
  'BFO:0000004',
  'CARO:0000000'],
 6: ['CL:0000226',
  'CARO:0030000',
  'CL:0000738',
  'CL:0000842',
  'BFO:0000004',
  'BFO:0000002'],
 7: ['CL:0000219',
  'CL:0000226',
  'CL:0000738',
  'CL:0000988',
  'BFO:0000004',
  'CARO:0000000',
  'CL:0002242',
  'BFO:0000002'],
 8: ['CL:0000219',
  'CARO:0030000',
  'CL:0002371',
  'CL:0000988',
  'CL:0000003',
  'CL:0000548',
  'CARO:0000000',
  'CL:0002242',
  'BFO:0000002'],
 9: ['CARO:0030000',
  'CL:0002371',
  'CL:0000000',
  'CL:0000003',
  'CL:0000548',
  'BFO:0000004',
  'CL:0000255'],
 10: ['BFO:0000002', 'CL:0000000', 'CL:0000003', 'B

## Get tree for all leafs

In [10]:
# from leaf_list only keep those in the cell ontology that start with CL

cl_leaf_list = [x for x in leaf_list if x.startswith('CL')]

len(cl_leaf_list)

1913

In [11]:

cl_leafs_tree = {}

for x in cl_leaf_list:
    cl_leafs_tree = get_tree(x,cl_leafs_tree,level=0)

# leafs_dict_test will have duplicates
# so let's remove them now

for key, value in cl_leafs_tree.items():
    cl_leafs_tree[key] = list(set(value))
        
cl_leafs_tree

{0: ['CL:0000965',
  'CL:1000322',
  'CL:1000356',
  'CL:0000982',
  'CL:0000337',
  'CL:0009062',
  'CL:0000258',
  'CL:0002675',
  'CL:2000024',
  'CL:0009110',
  'CL:0002102',
  'CL:0000647',
  'CL:0000042',
  'CL:0000853',
  'CL:0004238',
  'CL:0009053',
  'CL:0004225',
  'CL:0002285',
  'CL:0002594',
  'CL:0002046',
  'CL:0000615',
  'CL:4023031',
  'CL:0000352',
  'CL:0004244',
  'CL:4023164',
  'CL:0002538',
  'CL:4023026',
  'CL:0000418',
  'CL:0000783',
  'CL:0005017',
  'CL:0002530',
  'CL:1000304',
  'CL:1001107',
  'CL:2000057',
  'CL:4023104',
  'CL:4030012',
  'CL:1000397',
  'CL:0001075',
  'CL:0002345',
  'CL:1001097',
  'CL:0002524',
  'CL:0002244',
  'CL:4032000',
  'CL:0000032',
  'CL:0002585',
  'CL:0000544',
  'CL:4023181',
  'CL:1000358',
  'CL:4023055',
  'CL:4023052',
  'CL:0000950',
  'CL:4030025',
  'CL:1001005',
  'CL:4023094',
  'CL:0000480',
  'CL:0002367',
  'CL:0000695',
  'CL:0002383',
  'CL:0002289',
  'CL:0000110',
  'CL:0000534',
  'CL:4023098',
  'CL

In [14]:
# find duplicate values in dictionary

flipped = {}

for key, value in cl_leafs_tree.items():
    for x in value:
        if x not in flipped:
            flipped[x] = [key]
        else:
            flipped[x].append(key)

In [15]:
flipped

{'CL:0000965': [0],
 'CL:1000322': [0],
 'CL:1000356': [0],
 'CL:0000982': [0],
 'CL:0000337': [0],
 'CL:0009062': [0],
 'CL:0000258': [0],
 'CL:0002675': [0],
 'CL:2000024': [0],
 'CL:0009110': [0],
 'CL:0002102': [0],
 'CL:0000647': [0],
 'CL:0000042': [0],
 'CL:0000853': [0],
 'CL:0004238': [0],
 'CL:0009053': [0],
 'CL:0004225': [0],
 'CL:0002285': [0],
 'CL:0002594': [0],
 'CL:0002046': [0],
 'CL:0000615': [0],
 'CL:4023031': [0],
 'CL:0000352': [0],
 'CL:0004244': [0],
 'CL:4023164': [0],
 'CL:0002538': [0],
 'CL:4023026': [0],
 'CL:0000418': [0],
 'CL:0000783': [0],
 'CL:0005017': [0],
 'CL:0002530': [0],
 'CL:1000304': [0],
 'CL:1001107': [0],
 'CL:2000057': [0],
 'CL:4023104': [0],
 'CL:4030012': [0],
 'CL:1000397': [0],
 'CL:0001075': [0],
 'CL:0002345': [0],
 'CL:1001097': [0],
 'CL:0002524': [0],
 'CL:0002244': [0],
 'CL:4032000': [0],
 'CL:0000032': [0],
 'CL:0002585': [0],
 'CL:0000544': [0],
 'CL:4023181': [0],
 'CL:1000358': [0],
 'CL:4023055': [0],
 'CL:4023052': [0],


In [17]:
for key, value in flipped.items():
    if len(value) > 1:
        print(key, value)

CL:0000075 [1, 2, 3, 4, 5, 6, 7]
CL:0000069 [1, 2, 3]
CL:0002075 [1, 2]
CL:0000066 [1, 2, 3, 4, 5, 6, 7, 8]
CL:0002659 [1, 2, 3]
CL:2000032 [1, 2, 3]
CL:0002252 [1, 2]
CL:1001610 [1, 2]
CL:0000081 [1, 2, 3, 4, 5]
CL:0000120 [1, 3]
CL:1001433 [1, 2]
CL:0000019 [1, 2]
CL:0000412 [1, 3]
CL:1000617 [1, 2]
CL:0002405 [1, 2]
CL:0002393 [1, 3, 4]
CL:0002305 [1, 2, 3]
CL:0000817 [1, 2, 3, 4]
CL:0001035 [1, 2, 3, 4]
CL:0002494 [1, 2, 3, 4, 5, 6]
CL:0000080 [1, 2, 3]
CL:0000077 [1, 2]
CL:0000006 [1, 2, 3, 4, 5, 6]
CL:0000459 [1, 2]
CL:0000784 [1, 2]
CL:2000029 [1, 2, 3, 4, 5, 6]
CL:1000510 [1, 2, 3]
CL:0010003 [1, 2]
CL:0000841 [1, 2]
CL:0000034 [1, 2, 3, 4, 5, 6, 7]
CL:0000453 [1, 2]
CL:0011004 [1, 2]
CL:0000151 [1, 2, 3, 4, 5, 6, 7]
CL:0000150 [1, 2, 3, 4, 5]
CL:0001019 [1, 3, 4, 5]
CL:0000838 [1, 2]
CL:0000134 [1, 2, 3]
CL:0000679 [1, 2, 3, 4]
CL:4030034 [1, 2]
CL:0000161 [1, 2, 3, 4, 5, 6]
CL:1000507 [1, 3, 4, 5]
CL:0002608 [1, 2]
CL:0000598 [1, 2, 3, 4]
CL:0000035 [1, 2, 3, 4, 5]
CL:0000235

### Identify all Leaf Nodes

In [7]:
leaf_list = []

for term in cl.terms():
    if term.is_leaf():
        leaf_list.append(term.id)

In [8]:
leaf_list

['BFO:0000006',
 'BFO:0000019',
 'BFO:0000031',
 'BFO:0000034',
 'CARO:0000004',
 'CARO:0000010',
 'CARO:0000012',
 'CARO:0000013',
 'CARO:0000014',
 'CARO:0000032',
 'CARO:0000040',
 'CARO:0000042',
 'CARO:0000054',
 'CHEBI:14321',
 'CHEBI:15355',
 'CHEBI:15377',
 'CHEBI:15379',
 'CHEBI:15440',
 'CHEBI:16336',
 'CHEBI:16480',
 'CHEBI:16541',
 'CHEBI:16646',
 'CHEBI:16670',
 'CHEBI:16796',
 'CHEBI:16990',
 'CHEBI:16991',
 'CHEBI:17026',
 'CHEBI:17087',
 'CHEBI:17089',
 'CHEBI:17234',
 'CHEBI:17347',
 'CHEBI:17478',
 'CHEBI:17544',
 'CHEBI:18059',
 'CHEBI:18085',
 'CHEBI:18133',
 'CHEBI:18154',
 'CHEBI:18421',
 'CHEBI:22868',
 'CHEBI:23897',
 'CHEBI:23906',
 'CHEBI:23965',
 'CHEBI:24261',
 'CHEBI:24318',
 'CHEBI:24402',
 'CHEBI:24431',
 'CHEBI:24505',
 'CHEBI:24780',
 'CHEBI:24828',
 'CHEBI:24835',
 'CHEBI:24867',
 'CHEBI:24875',
 'CHEBI:24943',
 'CHEBI:25354',
 'CHEBI:25805',
 'CHEBI:25806',
 'CHEBI:25905',
 'CHEBI:26020',
 'CHEBI:26650',
 'CHEBI:26764',
 'CHEBI:27547',
 'CHEBI:27584',

In [7]:
# identify starting identifies of all leafs

unique_identifiers = []
             
for x in leaf_list:
    split_leaf = x.split(':')
    if split_leaf[0] not in unique_identifiers:
        unique_identifiers.append(split_leaf[0])

print(unique_identifiers)    

['BFO', 'CARO', 'CHEBI', 'CL', 'CP', 'http', 'GO', 'NCBITaxon', 'PATO', 'PR', 'RO', 'UBERON']


In [22]:
#cl.get_relationship("CL:0002116")
#cl["CL:0002116"].get_relationship()

## Identify subclasses of identifier

In [8]:
# print child terms for Leukocytes, within a distance of 2

root_node = cl['CL:0000934']

for term in root_node.subclasses(distance=1,with_self=False).to_set():
    print(term)

## Identify superclasses of identifier



In [9]:
root_node = cl['CL:0000934']

for term in root_node.superclasses(distance=3,with_self=False).to_set():
    print(term)

Term('CL:0000791', name='mature alpha-beta T cell')
Term('CL:0000624', name='CD4-positive, alpha-beta T cell')
Term('CL:0002419', name='mature T cell')
Term('CL:0000789', name='alpha-beta T cell')


In [20]:
root_node = cl['CL:0000202']

for term in root_node.superclasses(distance=1,with_self=False).to_set():
    print(term.id)

CL:0000855


In [27]:
def find_parent(subclass):
    root_node = cl[subclass]
    parents = []
    for term in root_node.superclasses(distance=1,with_self=False).to_set():
        parents.append(term.id)
    return parents

In [8]:
find_parent('CL:0000202')

['CL:0000855']

In [9]:
find_parent('CL:0000855')

['CL:0000199']

In [10]:
find_parent('CL:0000199')

['CL:0000101']

In [36]:
len(root_node.subclasses(distance=1,with_self=False).to_set())

2

In [39]:
list(root_node.subclasses(distance=1,with_self=False).to_set().ids)

['CL:0000589', 'CL:0000601']

In [61]:
# possible way to view, from page 10-11 of https://pronto.readthedocs.io/_/downloads/en/latest/pdf/

# Let's use the Vega radial tree example as a basis of the visualization
view = json.load(urllib.request.urlopen("https://vega.github.io/vega/examples/radial-tree-layout.vg.json"))

# First replace the default data with our own
view['data'][0].pop('url')
view['data'][0]['values'] = data

view['marks'][1]['encode']['enter']['tooltip'] = {"signal": "datum.desc"}
view['signals'][4]['value'] = 'cluster'
# Render the clustered tree
display({"application/vnd.vega.v5+json": view}, raw=True)

## Identify leafs under a particular cell ID

Idenfity the leafs under a particular cell ID. This will return a list of leaf cell ontologies.

In [5]:
root_node = cl['CL:0000542'] # lymphocyte

leaf_list = []

for term in root_node.subclasses(distance=None,with_self=False).to_set():
    if term.is_leaf():
        leaf_list.append(term.id)

In [6]:
leaf_list

['CL:0009018',
 'CL:0009113',
 'CL:0002106',
 'CL:0000935',
 'CL:0002048',
 'CL:0000949',
 'CL:0002126',
 'CL:0000805',
 'CL:0009060',
 'CL:0000951',
 'CL:0000986',
 'CL:0002039',
 'CL:0000969',
 'CL:0000902',
 'CL:0002514',
 'CL:0001048',
 'CL:0002051',
 'CL:0000794',
 'CL:0000958',
 'CL:0000963',
 'CL:0000901',
 'CL:0002409',
 'CL:0002108',
 'CL:0002345',
 'CL:0002050',
 'CL:0002432',
 'CL:0000917',
 'CL:0009082',
 'CL:0002343',
 'CL:0009050',
 'CL:0002415',
 'CL:0002348',
 'CL:0002040',
 'CL:0001045',
 'CL:0002446',
 'CL:0002124',
 'CL:0002444',
 'CL:0000916',
 'CL:0002440',
 'CL:0001041',
 'CL:0002218',
 'CL:0002125',
 'CL:0009037',
 'CL:0000937',
 'CL:0000973',
 'CL:0001075',
 'CL:0002127',
 'CL:0002042',
 'CL:4030030',
 'CL:0000900',
 'CL:0009032',
 'CL:0000907',
 'CL:0009083',
 'CL:0000964',
 'CL:0009069',
 'CL:0000913',
 'CL:0000976',
 'CL:2000006',
 'CL:0009111',
 'CL:0002403',
 'CL:0002423',
 'CL:0000821',
 'CL:0001202',
 'CL:0001046',
 'CL:0002448',
 'CL:0000918',
 'CL:00021